# 0.0. Imports

## 0.1. Libraries

In [4]:
# jupyter core
from IPython.core.display      import display, HTML
from IPython.display           import Image

# requests
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError

# scraping
from bs4 import BeautifulSoup

# data manipulation
import numpy as np
import pandas as pd
import re

# EDA
import matplotlib.pyplot as plt
import seaborn as sns

## 0.2. Helper Functions

In [27]:
def jupyter_settings():
    !matplotlib inline
    !pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]

    plt.rcParams['font.size'] = 12
    
    display(HTML("<style>.container { width:100% !important; }</style>"))
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()
    
def scrape_ads(url):    
    # scraping
    pages = 29

    # list, receiver of info from each renting ad, as a dictionaire
    cards = []

    for i in range(pages):
        # requesting html
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
        req = Request(url + str(i + 1), headers=headers)
        response = urlopen(req)
        html = response.read().decode('utf-8')

        # scraper
        soup = BeautifulSoup(html, 'html.parser')
        anuncios = soup.find('div', {"id": "column-main-content"}).findAll('div', class_="fnmrjs-2 jiSLYe")

        for anuncio in anuncios:
            card = {}

            # name
            card['NOME'] = anuncio.find("h2").getText()

            # value
            span_valor = anuncio.find("span", class_="sc-ifAKCX eoKYee")
            card["VALOR"] = span_valor.get_text() if span_valor else ''

            # details
            span_detalhe = anuncio.find("span", class_="sc-1j5op1p-0 lnqdIU sc-ifAKCX eLPYJb")
            card['DETALHES'] = span_detalhe.get_text().split('|') if span_detalhe else ''

            # address
            span_endereco = anuncio.find('span', class_="sc-7l84qu-1 ciykCV sc-ifAKCX dpURtf")
            card['ENDEREÇO'] = span_endereco.getText().split(', ')[1] if span_endereco else ''

            # appending list values to dict
            cards.append(card)    

    # saving csv (or json, )
    df_raw = pd.DataFrame(cards)
    return df_raw

In [8]:
jupyter_settings()

'matplotlib' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
'pylab' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## 0.3. Data Scraper

### Request test

In [5]:
# request test
url = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
try:
    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    print('OK')
except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)

OK


### Scraper

In [6]:
# scraping
pages = 29

# list, receiver of info from each renting ad, as a dictionaire
cards = []

for i in range(pages):
    # requesting html
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    req = Request('https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=' + str(i + 1), headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
    # scraper
    soup = BeautifulSoup(html, 'html.parser')
    anuncios = soup.find('div', {"id": "column-main-content"}).findAll('div', class_="fnmrjs-2 jiSLYe")
    
    for anuncio in anuncios:
        card = {}
        
        # name
        card['NOME'] = anuncio.find("h2").getText()
        
        # value
        span_valor = anuncio.find("span", class_="sc-ifAKCX eoKYee")
        card["VALOR"] = span_valor.get_text() if span_valor else ''
        
        # details
        span_detalhe = anuncio.find("span", class_="sc-1j5op1p-0 lnqdIU sc-ifAKCX eLPYJb")
        card['DETALHES'] = span_detalhe.get_text().split('|') if span_detalhe else ''
        
        # address
        span_endereco = anuncio.find('span', class_="sc-7l84qu-1 ciykCV sc-ifAKCX dpURtf")
        card['ENDEREÇO'] = span_endereco.getText().split(', ')[1] if span_endereco else ''
        
        # appending list values to dict
        cards.append(card)    
    
# saving csv (or json, )
df_raw = pd.DataFrame(cards)
df_raw.head()

,NOME,VALOR,DETALHES,ENDEREÇO
0,LC - Apartamento em Ponta Negra!,R$ 1.500,"[2 quartos , 58m² , 1 vaga]",Ponta Negra
1,"Alugo apartamento no Alecrim, sua casa no cent...",R$ 650,"[2 quartos , 50m²]",Alecrim
2,LCAP2201 - Ótimo apartamento em Tirol!,R$ 3.600,"[2 quartos , 200m² , 2 vagas]",Barro Vermelho
3,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio
4,Kitinet excelente localização Ponta Negra,R$ 2.500,"[1 quarto , 14m² , Condomínio: R$ 0]",Ponta Negra


In [21]:
df_raw.to_csv('df_raw.csv')

# Data Cleaning

In [7]:
df1 = df_raw.copy()

In [8]:
df1.shape

(1448, 4)

In [9]:
df1.dtypes

NOME        object
VALOR       object
DETALHES    object
ENDEREÇO    object
dtype: object

### Regex em DETALHES

In [10]:
# lambda functions and mapping the column DETALHES to separate each section
f_quarto = lambda x: re.findall('[0-9]+ quartos?', str(x))
f_area = lambda x: re.findall('[0-9]+m²', str(x))
f_condominio = lambda x: re.findall('Condomínio: R\$\ [0-9]+', str(x))
f_vaga = lambda x: re.findall('[0-9]+ vagas?', str(x))

df1['QUARTOS'] = list(map(f_quarto, df1['DETALHES']))
df1['AREA'] = list(map(f_area, df1['DETALHES']))
df1['CONDOMINIO'] = list(map(f_condominio, df1['DETALHES']))
df1['VAGAS'] = list(map(f_vaga, df1['DETALHES']))

df1.head()

,NOME,VALOR,DETALHES,ENDEREÇO,QUARTOS,AREA,CONDOMINIO,VAGAS
0,LC - Apartamento em Ponta Negra!,R$ 1.500,"[2 quartos , 58m² , 1 vaga]",Ponta Negra,[2 quartos],[58m²],[],[1 vaga]
1,"Alugo apartamento no Alecrim, sua casa no cent...",R$ 650,"[2 quartos , 50m²]",Alecrim,[2 quartos],[50m²],[],[]
2,LCAP2201 - Ótimo apartamento em Tirol!,R$ 3.600,"[2 quartos , 200m² , 2 vagas]",Barro Vermelho,[2 quartos],[200m²],[],[2 vagas]
3,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio,[],[],[],[]
4,Kitinet excelente localização Ponta Negra,R$ 2.500,"[1 quarto , 14m² , Condomínio: R$ 0]",Ponta Negra,[1 quarto],[14m²],[Condomínio: R$ 0],[]


In [11]:
# lambda function and mapping to extract the number to count each feature
f1_quarto = lambda x: re.findall('[0-9]+', str(x))
f1_area = lambda x: re.findall('[0-9]+', str(x))
f1_condominio = lambda x: re.findall('[0-9]+', str(x))
f1_vaga = lambda x: re.findall('[0-9]+', str(x))

df1['QUARTOS'] =  list(map(f1_quarto, df1['QUARTOS']))
df1['AREA'] = list(map(f1_area, df1['AREA']))
df1['CONDOMINIO'] = list(map(f1_condominio, df1['CONDOMINIO']))
df1['VAGAS'] = list(map(f1_vaga, df1['VAGAS']))

df1.head()

,NOME,VALOR,DETALHES,ENDEREÇO,QUARTOS,AREA,CONDOMINIO,VAGAS
0,LC - Apartamento em Ponta Negra!,R$ 1.500,"[2 quartos , 58m² , 1 vaga]",Ponta Negra,[2],[58],[],[1]
1,"Alugo apartamento no Alecrim, sua casa no cent...",R$ 650,"[2 quartos , 50m²]",Alecrim,[2],[50],[],[]
2,LCAP2201 - Ótimo apartamento em Tirol!,R$ 3.600,"[2 quartos , 200m² , 2 vagas]",Barro Vermelho,[2],[200],[],[2]
3,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio,[],[],[],[]
4,Kitinet excelente localização Ponta Negra,R$ 2.500,"[1 quarto , 14m² , Condomínio: R$ 0]",Ponta Negra,[1],[14],[0],[]


### Cleaning Helper Functions

In [12]:
def list_quartos(n):
    # lista com numeros de quartos
    lst = []
    
    # iterando as strings com valores de numeros de quarto
    for i in n:
        if len(i)>3:
            lst.append(i[-3])
        else:
            lst.append('1') #valores vazios são interpretados como kitnets e insere-se 1 quarto
    
    # convertendo o valor numerico de str para numero
    lst = pd.Series(pd.to_numeric(lst))
    
    return lst

def list_vagas(n):
    # lista com numeros de vagas
    lst = []
    
    # iterando as strings com valores de numeros de quarto
    for i in n:
        if len(i)>3:
            lst.append(i[-3])
        else:
            lst.append('0') #valores vazios são interpretados como 0 vaga
    
    # convertendo o valor em str para numerico int
    lst = pd.Series(pd.to_numeric(lst))
    
    return lst

def list_cond(n):
    # lista com valores de condominio
    lst = []
    
    # iterando as strings com valores de condominio
    for i in n:
        if len(i)>2:
            lst.append(''.join(re.findall('[0-9]+', i)))
        else:
            lst.append(0)
    
    # convertendo os valores em str para numerico int
    lst = pd.Series(pd.to_numeric(lst))
    
    return lst

def list_area(n):
    # lista com valores de area
    lst = []
    
    # iterando as strings com valores de area
    for i in n:
        if len(i)>2:
            lst.append(''.join(re.findall('[0-9]+', i)))
        else:
            lst.append(0)
    
    # convertendo os valores em str para numerico int
    lst = pd.Series(pd.to_numeric(lst))
    
    return lst

### Quartos

In [13]:
list_quartos(df1['QUARTOS'].astype(str))

0       2
1       2
2       2
3       1
4       1
       ..
1443    3
1444    2
1445    1
1446    4
1447    2
Length: 1448, dtype: int64

### Vagas

In [14]:
list_vagas(df1['VAGAS'].astype(str))

0       1
1       0
2       2
3       0
4       0
       ..
1443    2
1444    1
1445    0
1446    3
1447    0
Length: 1448, dtype: int64

### Condominio

In [15]:
list_cond(df1['CONDOMINIO'].astype(str))

0       0
1       0
2       0
3       0
4       0
       ..
1443    0
1444    0
1445    0
1446    0
1447    0
Length: 1448, dtype: int64

### Área

In [16]:
list_area(df1['AREA'].astype(str))

0        58
1        50
2       200
3         0
4        14
       ... 
1443      0
1444     46
1445     18
1446    176
1447     57
Length: 1448, dtype: int64

# Data Description

In [19]:
df2 = pd.DataFrame()

df2['NOME'] = df1['NOME']
df2['ENDERECO'] = df1['ENDEREÇO']
df2['QUARTOS'] = list_quartos(df1['QUARTOS'].astype(str))
df2['VAGAS'] = list_quartos(df1['VAGAS'].astype(str))
df2['CONDOMINIO'] = list_quartos(df1['CONDOMINIO'].astype(str))
df2['AREA'] = list_quartos(df1['AREA'].astype(str))

df2.head()

,NOME,ENDERECO,QUARTOS,VAGAS,CONDOMINIO,AREA
0,LC - Apartamento em Ponta Negra!,Ponta Negra,2,1,1,8
1,"Alugo apartamento no Alecrim, sua casa no cent...",Alecrim,2,1,1,0
2,LCAP2201 - Ótimo apartamento em Tirol!,Barro Vermelho,2,2,1,0
3,Kitnet UFRN cidade jardim capim macio,Capim Macio,1,1,1,1
4,Kitinet excelente localização Ponta Negra,Ponta Negra,1,1,0,4


In [20]:
df2.to_csv('df_clean.csv')

# EDA

In [26]:
df3 = df2.copy()